In [ ]:
#Mount Google Drive with this Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from astropy.io import fits
import astropy
import glob,os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy import units as u
from astropy.nddata import Cutout2D
import datetime
from PIL import Image
from astropy.visualization import MinMaxInterval
from astropy.visualization import PercentileInterval
from astropy.stats import sigma_clipped_stats
from tqdm import tqdm

import astropy
from astropy.io import fits
from astropy.visualization import AsinhStretch, MinMaxInterval
from PIL import Image, ImageOps

from astropy.table import Table

In [ ]:
def create_cutouts(image,galaxy,ra,dec,size,size_factor=3,overwrite_fits=True):

    if not os.path.exists("fits_euclid"):
        os.makedirs("fits_euclid")

    try:
        objCoord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree)

        s=size_factor*size

        FoV = np.array([s, s])
        FoV_dimen = u.Quantity((FoV[0], FoV[1]), u.arcsec)

        cutout = Cutout2D(image, objCoord, FoV_dimen, wcs=wcs, mode='strict')
        wcs_cut = cutout.wcs
        hdr.update(wcs_cut.to_header())
        hdr['COMMENT'] = "= Cropped fits file ({}).".format(datetime.date.today())
        img = cutout.data
        img = Image.fromarray(img)
        im = img.resize((300,300))
        resized_data = np.array(im)
        fits.writeto("fits_euclid/"+str(galaxy)+".fits", resized_data, overwrite=True)
        print("Successful.")

    except Exception as e:
        print("Error -",e,str(galaxy))

In [ ]:
hdul = fits.open("/content/drive/MyDrive/MPE/2022_Ben_Aussel/Data/Euclidized_COSMOS/Data Challenge 2/sim_vis_DC2/sim_vis_cosmos_1.2deg.fits")
image = hdul[0].data
hdr = hdul[0].header
hdul.close()

wcs = WCS(hdul[0].header)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.


In [ ]:
data_complete = pd.read_csv('/content/drive/MyDrive/MPE/ZoobotEuclid/Data/Hubble_COSMOS_labels_complete.csv')
data = data_complete
data

In [ ]:
for i in tqdm(range(0,len(data))):
    create_cutouts(image,data["ObjNo"].iloc[i], data["RA"].iloc[i], data["DEC"].iloc[i], data["rKron1"].iloc[i],3)

In [ ]:
!zip -r /content/fits_euclid_complete.zip fits_euclid

In [ ]:
!cp fits_euclid_complete.zip /content/drive/MyDrive/MPE/ZoobotEuclid/Data/Euclidized_COSMOS_complete/